In [1]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Add an index value to each row
import pandas as pd

# File paths
input_path = "kiba_matrix_triplets.csv"  # Input file path
output_path = "kiba_matrix_triplets_processed.csv"  # Output file path

# Read data
data = pd.read_csv(input_path, header=None)
data.columns = ["DrugID", "ProteinID", "Affinity"]  # Add column names

# Fill NaN values with -5
data.fillna(-5, inplace=True)

# Add a new column containing "# and the corresponding row index"
data["NewColumn"] = data.index.map(lambda x: f"#{x}")

# Save the result to a new file
data.to_csv(output_path, index=False)

print(f"The processed data has been saved to {output_path}.")


处理后的数据已保存到 kiba_matrix_triplets_processed.csv 文件中。


In [3]:
# Remove rows where the Affinity value is -5
import pandas as pd

# File paths
input_path = "kiba_matrix_triplets_processed.csv"  # Input file path
output_path = "kiba_matrix_triplets_filtered.csv"  # Output file path

# Read data
data = pd.read_csv(input_path)

# Remove rows where the value in the Affinity column is -5
filtered_data = data[data["Affinity"] != -5]

# Save the remaining data to a new file
filtered_data.to_csv(output_path, index=False)

print(f"The processed data has been saved to {output_path}.")


处理后的数据已保存到 kiba_matrix_triplets_filtered.csv 文件中。


In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
import os

# Read data
file_path = "kiba_matrix_triplets_filtered.csv"
data = pd.read_csv(file_path)

# Initialize 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Group by DrugID
grouped = data.groupby("DrugID")

# Cross-validation output directory and file templates
output_dir = "./cross_validation_sets/"
train_file_template = output_dir + "train_indices_fold_{}.csv"
test_file_template = output_dir + "test_indices_fold_{}.csv"

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over each fold in cross-validation
for fold, (train_idx, test_idx) in enumerate(kf.split(data)):
    train_file_path = train_file_template.format(fold + 1)
    test_file_path = test_file_template.format(fold + 1)
    
    # Save training set
    train_indices = []
    for drug_id, group in grouped:
        train_indices += group.index.intersection(train_idx).tolist()
    train_data = data.loc[train_indices, ["NewColumn"]]
    train_data["Index"] = train_data.index
    train_data = train_data[["Index", "NewColumn"]]
    train_data.to_csv(train_file_path, index=False)
    
    # Save testing set
    test_indices = []
    for drug_id, group in grouped:
        test_indices += group.index.intersection(test_idx).tolist()
    test_data = data.loc[test_indices, ["NewColumn"]]
    test_data["Index"] = test_data.index
    test_data = test_data[["Index", "NewColumn"]]
    test_data.to_csv(test_file_path, index=False)
    
    print(f"Fold {fold + 1}:")
    print(f"  Training set saved to {train_file_path}")
    print(f"  Testing set saved to {test_file_path}")


In [10]:
# Batch process all files in the cross_validation_sets/ folder:
# Extract the number after '#' in the NewColumn column, sort in ascending order,
# and output to the corresponding file.

import os
import pandas as pd

# Folder paths
input_folder = "cross_validation_sets/"
output_folder = "train_test_fold/"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get all file names
file_list = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# Batch process files
for file_name in file_list:
    input_path = os.path.join(input_folder, file_name)
    output_path = os.path.join(output_folder, file_name)
    
    # Read the file
    data = pd.read_csv(input_path)
    
    # Extract the numeric part after '#' in the NewColumn column
    data['Processed'] = data['NewColumn'].str.extract(r'#(\d+)').astype(int)
    
    # Sort in ascending order
    sorted_data = data['Processed'].sort_values().reset_index(drop=True)
    
    # Output to file
    sorted_data.to_csv(output_path, index=False, header=False)
    
    print(f"Processing completed: {output_path}")


处理完成: train_test_fold/train_indices_fold_1.csv
处理完成: train_test_fold/test_indices_fold_1.csv
处理完成: train_test_fold/train_indices_fold_2.csv
处理完成: train_test_fold/test_indices_fold_2.csv
处理完成: train_test_fold/train_indices_fold_3.csv
处理完成: train_test_fold/test_indices_fold_3.csv
处理完成: train_test_fold/train_indices_fold_4.csv
处理完成: train_test_fold/test_indices_fold_4.csv
处理完成: train_test_fold/train_indices_fold_5.csv
处理完成: train_test_fold/test_indices_fold_5.csv
